In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer
import torch

ModuleNotFoundError: No module named 'transformers'

In [ ]:
# from unsloth import FastLanguageModel
# from transformers import TrainingArguments
# from trl import SFTTrainer  # Unsloth still uses this under the hood

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
# Load tokenizer and model
model_id = "/home/siamai/data/huggingface/hub/models--scb10x--llama3.1-typhoon2-8b-instruct/snapshots/30e8c65ac39307c9a4e3fb46a444db5745214516"
dataset_id = "/home/siamai/data/huggingface/hub/datasets--Josephgflowers--Finance-Instruct-500k/snapshots/379407b4708ededdf48cd33d1e1cffda45cc56f4"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Important for causal LM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

In [ ]:
# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
# Apply LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Load dataset
dataset = load_dataset(dataset_id, split="train[:20%]")
len(dataset)

In [ ]:
dataset

In [ ]:
# Preprocessing: convert "instruction", "input", "output" to prompt-completion format
def format_prompt(example):
    if example["user"].strip():
        prompt = f"### Instruction:\n{example['system']}\n\n### Input:\n{example['user']}\n\n### Response:\n{example['assistant']}"
    else:
        prompt = f"### Instruction:\n{example['user']}\n\n### Response:\n{example['assistant']}"
    return {"text": prompt}

dataset = dataset.map(format_prompt)
dataset = dataset.train_test_split(test_size=0.01)

# Tokenization
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset["train"].column_names)

In [ ]:
tokenized_dataset

In [ ]:
# # Training arguments
# training_args = TrainingArguments(
#     output_dir="./llama3-finance-lora",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     gradient_accumulation_steps=4,
#     eval_strategy="steps",
#     eval_steps=100,
#     logging_steps=50,
#     save_steps=200,
#     save_total_limit=2,
#     num_train_epochs=3,
#     fp16=True,
#     logging_dir="./logs",
#     report_to="none"
# )

# # SFT Trainer
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
#     args=training_args
# )

# trainer.train()

In [ ]:
# Load 8-bit model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 4096,
    dtype = "auto",             # automatically chooses fp16 or bf16
    load_in_8bit = True         # or load_in_4bit = True
)

# Add LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # suitable for LLaMA models
    bias="none",
    task_type="CAUSAL_LM"
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama3-finance-lora",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    num_train_epochs=3,
    fp16=True,
    logging_dir="./logs",
    report_to="none"
)

# SFTTrainer from TRL (still used in Unsloth)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
)

# Train
trainer.train()


ModuleNotFoundError: No module named 'xformers'